In [1]:
from ase.io import read, write
from mace.calculators import mace_mp
from ase.build import surface
from ase.visualize import view
from ase.constraints import FixAtoms, UnitCellFilter
from ase.build import bulk
import numpy as np
from ase.optimize import LBFGS

x = read('/home/lana/Downloads/9007497.cif')
x.calc = mace_mp(model="medium", dispersion=False, default_dtype="float64", device='cpu')
ucf = UnitCellFilter(x)
qn = LBFGS(ucf, trajectory='bulk_rlx.traj')
qn.run(fmax=0.01)
E_x = x.get_potential_energy()
t = read('bulk_rlx.traj@:')
view(t)

Using Materials Project MACE for MACECalculator with /home/lana/.cache/mace/5yyxdm76
Using float64 for MACECalculator, which is slower but more accurate. Recommended for geometry optimization.
       Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
LBFGS:    0 10:42:52      -28.749845*       0.0015


<Popen: returncode: None args: ['/home/lana/ve/bin/python3', '-m', 'ase', 'g...>

In [29]:
from ase import Atoms
from mace.calculators import mace_mp
from ase.build import surface
from ase.constraints import FixAtoms, UnitCellFilter
from ase.optimize import QuasiNewton
from ase.visualize import view
from ase.io import read, write
from ase.optimize import LBFGS
import numpy as np


def cu2o_bulk():
  return read('bulk_rlx.traj')

b = cu2o_bulk()
b.calc = mace_mp(model="medium", dispersion=False, default_dtype="float64", device='cpu')
E_bulk = b.get_potential_energy()


def cu2o111(n_slab, vacuum):
  x = cu2o_bulk()
  x.translate([0.1, 0.1, 0])
  slab = surface(x, (1,1,1), n_slab, vacuum=vacuum, periodic=True)  
  return slab 


n_layers = 2
slab = cu2o111(n_layers, 10)

#view(slab)
write('slab.xyz', slab)


slab.calc = mace_mp(model="medium", dispersion=False, default_dtype="float64", device='cpu')

E_slab = slab.get_potential_energy()
E_cleav = (E_slab - E_bulk * n_layers) / 2 / np.linalg.det(slab.cell[:2, :2])
print(f'{n_layers=} {E_cleav=}')

bottom_Cu_z = np.min(slab[slab.symbols=='Cu'].positions[:,2])
slab.set_constraint(FixAtoms(mask=slab.positions[:, 2] < bottom_Cu_z + 1.0))
qn = LBFGS(slab, trajectory='111slab.traj')
qn.run(fmax=0.01)
E_slab = slab.get_potential_energy()
t = read('111slab.traj@:')
atoms = t[-1]
#view(t)

del atoms[atoms.positions[:, 2] > 14.0]
view(atoms)



   
E_surf = (E_slab - E_bulk * n_layers) / 2 / np.linalg.det(slab.cell[:2, :2])
print(f'{n_layers=} {E_surf=}')


Using Materials Project MACE for MACECalculator with /home/lana/.cache/mace/5yyxdm76
Using float64 for MACECalculator, which is slower but more accurate. Recommended for geometry optimization.
Using Materials Project MACE for MACECalculator with /home/lana/.cache/mace/5yyxdm76
Using float64 for MACECalculator, which is slower but more accurate. Recommended for geometry optimization.
n_layers=2 E_cleav=0.1000247721074349
       Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
LBFGS:    0 12:47:19      -51.152397*       1.3559
LBFGS:    1 12:47:19      -51.230513*       1.1450
LBFGS:    2 12:47:19      -51.497881*       0.6783
LBFGS:    3 12:47:20      -51.642747*       0.5408
LBFGS:    4 12:47:20      -51.675589*       0.5726
LBFGS:    5 12:47:20      -51.697492*       0.5045
LBFGS:    6 12:47:20      -51.769656*       0.4229
LBFGS:    7 12:47:21      -51.833494*       0.5629
LBFGS:    8 12:47:21      -51.865681*       0.7308
LBFGS:    9 12:47: